# Advanced Data Science Capstone

## Air pollution and prevalence of bronchial asthma in Germany

## Deep learning algorithm for analysis of disease risks in counties based on available air pollution monitoring data

### The aim

  - The aim of the present study is creating an algorithm allowing identification of elevated-risk areas for certain diseases
based on analysis of public available air pollution monitoring sensors data. 
  - As an example the prevalence of bronchial asthma over German counties is considered.

### Data Sources 


  -  The officially published data sets by **Geschäfts- und Koordinierungsstelle GovData**, the search engine is available at https://www.govdata.de/web/guest/suchen.
  - Data stream **E1a** contains measured (Link to Data stream **D**) values of gas phase pollutants (e.g. Ozone, NO2, SO2, CO), particle pollutants (e.g. dust) and dust constituants (e.g. heavy metals, PAK in PM10, PM2.5, TSP) as well es total deposition (BULK), wet deposition and meteorologic data (e.g. temperature, wind, pressure)for every measurement location.
  - The data for years 2013 - 2018 is currently available. For the project I will limit myself with 2016 data (due to limited availability of the health related data sets), however the method and the model are easily extendable for the data for other years.
  - Compressed dataset is available at https://datahub.uba.de/server/rest/directories/arcgisforinspire/INSPIRE/aqd_MapServer/Daten/AQD_DE_E1a_2016.zip.
  - For details see *Data Exploration* document (https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/48bc8b77-5a2c-44e2-82d3-8df365d1dfe0/view?access_token=06ee7b733563e5c2301dbe2b935927b3324a59101eb41660b7879938b8b87b0b).
  
### Data cleansing and Enterprise data storage
 
  - The air quality data sets are claimed to be "validated", so most work for cleansing the data is already done.
  - The incomplete files from the datasets (not having "hour" in the name) are ignored.
  - Few missing values (below 10%) appearing in the time series as negative values of the pollutant concentrations will be imputed.
  - The sensors with heavily corrupted data (above 10% of measurements) will be dropped.
  - The total data loss due to the procedures mentioned above is below 2%.
  - Saving Spark data frames, containing hourly averaged values of pollutants concentration, pollutant type and ID of the county for all available high-quality measurements; disease prevalence vs ID of the county Apache Spark DataFrames to the Cloud Object Storage (COS) in the Parquet format.
  - For details see *ETL* document (https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/ff119023-c24d-44cb-80d0-ce413c61f980/view?access_token=8c4e8aaeed256ec484dc24e7559accef62fedd414b72a5da964a8589221f94df). 
 
### Feature Engineering
  - A number of approaches to Feature creation have been tested:
    - three or five selected pollutants to be kept in the feature set
    - mean values, values of 50th, 75th or 95th percentile of the pollutant concentration
    - number of hours when the pollutant concentration was exceeding some certain value
    - polynomial features, containing powers of selected pollutants concentrations in powers up to 2
    - principal component analysis (PCA)
    - percentile-based features for disease prevalence: the high risk county flag is set, when the county is in 50th, 75th or 95th percentile of bronchial asthma prevalence over the counties.
  - The best feature set based on evaluation on the set of models is **dfPolMeanLongDisease95perc**, containing the mean values of the limited pollutant set (NO, NO2, PM1) over the year and presence of the county in 95th percentile of bronchial asthma prevalence.
  - For examples of Feature sets creation and cross-correlation analysis see *Pandas Feature Engineering* document (https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/74c21755-9e18-48f8-b874-8997909c75da/view?access_token=c6fd9286983738040e16fb3eee18ede50f7760661cfca18406d81f650e13289b). Please note, that alternative version of *ETL* is needed for this document, for details see *ETL Pandas* (https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/a41dd4e7-0cf9-4cda-9270-0879ebcd6bd7/view?access_token=93bdd4f8e25df9d8f6ea4ec7e5c472db2df3a30a3a9544932945b9a389e575ce).
  - The final version of feature creation document by means of Apache Spark SQL is available at https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/f6607eb8-7da7-484a-9699-406ef4fa7cc2/view?access_token=8ed45d076a87d2703acfe5afe97b15e2b3571df8d6538a7816180e73a02becff.
  
### Model definition
  A number of approaches available in the framework of *Apache SparkML* were tested on available data sets:
  - Logistic Regression (LR)
  - Gradient-Boosted Trees (GBTs)
  - Linear Support Vector Machine Classifier (LSVC)
  - Deep learning Multilayer Perceptron Classifier (MLPC)
    
### Model training and evaluation
  - The models were trained and evaluated on the 2016 year data for air pollution and bronchial asthma prevalence.
  - The available data was splitted into train and test data sets as 80% and 20%.
  - The best model found to be *Deep learning Multilayer Perceptron Classifier (MLPC)*, providing nearly 100% accuracy on *dfPolMeanLongDisease95perc* feature set.
  - The evaluation of the models on six feature sets defined in the *Features SQL* document(https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/f6607eb8-7da7-484a-9699-406ef4fa7cc2/view?access_token=8ed45d076a87d2703acfe5afe97b15e2b3571df8d6538a7816180e73a02becff) are available at:
    - Logistic Regression (*reproducible* pattern, ETL to evaluation in one notebook): https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/6761eaba-bb1a-4ba7-83e4-ec010eea5ce7/view?access_token=929140ca73cfae6462d0e15118257a5789ee490df35ebd0a0cc9ef636df65283  
    - Gradient-Boosted Trees (GBTs): https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/a36f6e0d-ad74-4847-ae3a-4be651c7362f/view?access_token=41f3645070aa5f0b433450a04c44859d6a6ccb6c02852fb859f1aa47472562b4
    - Linear Support Vector Machine Classifier (LSVC): https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/e92c7084-6bad-4037-8048-e219bb4530b0/view?access_token=6b58d3be1c87216a1a18dc4bb7fa5c5f3edd0f3695a6f7021f2ce098b9bebcfe
    - Deep learning Multilayer Perceptron Classifier (MLPC): https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/c242e639-7c14-4b69-bf56-9337e6e1e416/view?access_token=7a013b4fe6f0d5ac6b2b2d33e8001829db1ae11da265d189aa4f40c4b270d229
  
### Use case

Construction of a general framework, allowing to analyze and predict the prevalence of certain diseases in selected areas,
identifying counties with elevated risk of the asthma disease based on available data of pollutant concentrations.
The available data for 2016 is taken: hourly concentration of pollutants measured by sensors distributed along the Germany; prevalence of Bronchial asthma in German counties in 2016.
The best feature set as demonstrated on *model evaluation* stage (*dfPolMeanLongDisease95perc*, containing the mean values of the limited pollutant set (NO, NO2, PM1) over the year and presence of the county in 95th percentile of bronchial asthma prevalence) has been selected. The best classification method as demonstrated on *model evaluation* stage (Deep learning Multilayer Perceptron Classifier, MLPC) has been chosen.

### Algorithm demonstration

The Feature set **dfPolMeanLongDisease95perc** is loaded into **df** Apache Spark DataFrame from the Cloud Object Storage (for details of the Feature set creation see *Features SQL* document(https://eu-de.dataplatform.cloud.ibm.com/analytics/notebooks/v2/f6607eb8-7da7-484a-9699-406ef4fa7cc2/view?access_token=8ed45d076a87d2703acfe5afe97b15e2b3571df8d6538a7816180e73a02becff). The DataFrame structure is:

In [1]:
# The code was removed by Watson Studio for sharing.

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190829105743-0001
KERNEL_ID = 804e95b6-0b8a-4e9c-aaf2-023f6e2310b3


In [2]:
df.show(3)

+--------+------------+------------------+------------------+------------------+
|CountyID|DiseaseRFeat|                NO|               NO2|               PM1|
+--------+------------+------------------+------------------+------------------+
|    1003|           0| 9.739761839708535|21.588235200364284|15.184396971766851|
|    1057|           0|1.8715591416211297| 9.932119649362456|14.159970913023711|
|   16053|           0|  6.65648185440208|16.939192214783706|16.377837658670753|
+--------+------------+------------------+------------------+------------------+
only showing top 3 rows



Preparing the model pipeline, containing *indexer*, *vectorAssembler*, *normalizer* steps converting the Feature set to the most appropriate form for the Machine Learning algorithms.
The heart of the algorithm is **classifier**, in the present case it is a deep learning Multilayer Perceptron Classifier (MLPC). Configuration of the perceptron is defined by *layers = [3, 8, 7, 2]* variable, that denotes a four-layer perceptron with 

 - input layer of size 3 (number of input features: NO, NO2 and PM1 concentrations)
 - two intermediate layers of size 8 and 7
 - output layer of size 2 (output classes: elevated risk of asthma Yes/No)

In [3]:
indexer = StringIndexer(inputCol="DiseaseRFeat", outputCol="label")
vectorAssembler = VectorAssembler(inputCols=["NO", "NO2", "PM1"], outputCol="featuresOO")
normalizer = Normalizer(inputCol="featuresOO", outputCol="features", p=1.0)

layers = [3, 8, 7, 2]

classifier = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=200, seed=1234)
classifier.setLabelCol('label')
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, classifier])

Demonstration of the algorithm is done a following way: the feature set is being randomly splitted 20 times into the *train* and *test* data sets in proportion 80%/20%;
for each set the model is being trained within the *train* set and tested using corresponding *test* set:

In [4]:
for Splitting in range(0, 20):
    splits = df.randomSplit([0.8, 0.2])
    df_train = splits[0]
    df_test  = splits[1]
    model = pipeline.fit(df_train)
    prediction = model.transform(df_train)
    binEval = MulticlassClassificationEvaluator().setMetricName("accuracy").setPredictionCol("prediction").setLabelCol("DiseaseRFeat")
    InSampleAcc = binEval.evaluate(prediction)
    predictionTest = model.transform(df_test)
    OutOfSampleAcc = binEval.evaluate(predictionTest)  
    Eval = Eval.union(spark.createDataFrame([(InSampleAcc,OutOfSampleAcc)], accuracies))

Now one can estimate the in-sample accuracy (accuracy demonstrated on the *train* set used for training) ant out-of-sample accuracy (accuracy demonstrated on the *test* set unseen by the network during the training).

In [5]:
Eval.show()

+------------------+-------------------+
|  InSampleAccuracy|OutOfSampleAccuracy|
+------------------+-------------------+
|               1.0|                0.4|
| 0.926829268292683|              0.875|
|0.9459459459459459| 0.8333333333333334|
|0.9210526315789473| 0.8181818181818182|
|0.8857142857142857| 0.9285714285714286|
|0.9210526315789473| 0.9090909090909091|
|0.9523809523809523| 0.8571428571428571|
|               1.0| 0.6923076923076923|
|0.9473684210526315| 0.9090909090909091|
|               0.9|                1.0|
|            0.9375| 0.7647058823529411|
|0.9736842105263158| 0.7272727272727273|
|0.9736842105263158|                1.0|
|0.9302325581395349|                1.0|
|               1.0|               0.75|
|0.9743589743589743|                0.8|
| 0.926829268292683|               0.75|
|               1.0| 0.8181818181818182|
|             0.925| 0.7777777777777778|
| 0.918918918918919| 0.9166666666666666|
+------------------+-------------------+



The average out-of-sample accuracy, i.e. correctly predicted risk status of the county based on the air pollution monitoring data is

In [7]:
Eval.agg({"OutOfSampleAccuracy": "avg"}).show()

+------------------------+
|avg(OutOfSampleAccuracy)|
+------------------------+
|       0.826366190998544|
+------------------------+



### Conclusions
The algorithm allowing identification of elevated-risk areas for certain diseases based on analysis of public available air pollution monitoring sensors data is created
and tested on the air pollution data and prevalence of bronchial asthma 2016. The quality of the algorithm can be improved using more data for the training set, e.g. air pollution data for 2017 and 2018 contains considerable more information. However the health indicator data is being updated much slower (2016 is the last available year for the bronchial asthma prevalence), so another approaches, like cross-validation, and possible use of consequent years data.